<a href="https://colab.research.google.com/github/priyal6/gen/blob/main/MediBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U sentence-transformers #Ensure the sentence_transformers library is installed and updated.

In [5]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 59.3 MB/s eta 0:00:00


In [15]:
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
import numpy as np


tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Function to generate embeddings
def get_embeddings(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the CLS token as the sentence embedding
    return outputs.last_hidden_state[:, 0, :].numpy()

#docs
medical_docs = [
    "Fever and fatigue are symptoms of COVID-19.",
    "Shortness of breath is a common symptom of asthma.",
    "Chest pain can indicate a heart attack."
]

#embeddings
embeddings = get_embeddings(medical_docs)


In [17]:
# FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)


In [18]:
# Query embedding
query = "What causes shortness of breath?"
query_embedding = get_embeddings([query])

# FAISS index
k = 2
distances, indices = index.search(query_embedding, k)
retrieved_docs = [medical_docs[i] for i in indices[0]]

print("Retrieved Documents:")
print(retrieved_docs)


Retrieved Documents:
['Shortness of breath is a common symptom of asthma.', 'Chest pain can indicate a heart attack.']
